<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=bd64e75465214d4c9f224f9423437dc19fc3d407b9d8bf4e137ced33388de458
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-5*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

mypf = mypf[mypf['InPortfolio'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()
stock_ids

Mounted at /content/drive


array(['5PAISA', 'AAVAS', 'AMARAJABAT', 'AMBUJACEM', 'ASIANPAINT',
       'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BATAINDIA', 'BAYERCROP',
       'DMART', 'FINEORG', 'GICRE', 'GILLETTE', 'GLAND', 'HDFCAMC',
       'ICICIGI', 'INDIGOPNTS', 'INDUSINDBK', 'INFY', 'IRCTC',
       'JMFINANCIL', 'JUBLFOOD', 'LUXIND', 'NIACL', 'PAGEIND',
       'PIDILITIND', 'PNB', 'RELAXO', 'RELIANCE', 'TEAMLEASE',
       'UJJIVANSFB', 'VAIBHAVGBL', 'VALIANTORG', 'WIPRO'], dtype=object)

In [5]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='5y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    stock_df['Stock'] = stock_symbol
    
    stock_df['Close'] = round(stock_df['Close'],0)     
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    #current_pe = get_current_pe(stock_id)
    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp['Prev_Close'] = tmp['Close'].shift(1)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]   
    tmp['Criteria'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Criteria'].values[0]   
    tmp['Strategy'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Strategy'].values[0]   
    #tmp['Curr_PE'] = current_pe
    df_prec_dev = df_prec_dev.append(tmp)

In [6]:
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

In [7]:
# portfolio stocks

tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df['Investment'] = tmp_df['Avg_Cost'] * tmp_df['Shares']
tmp_df['Current'] = tmp_df['Close'] * tmp_df['Shares']
tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
tmp_df['Estimated'] = tmp_df['Max'] * tmp_df['Shares']
tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Estimated P/L%'] = round((tmp_df['Estimated'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
tmp_df['Juice Left%'] = tmp_df['Estimated P/L%'] - tmp_df['Current P/L%'] 

investment = round(sum(tmp_df['Avg_Cost']*tmp_df['Shares']),0)
current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
previous = round(sum(tmp_df['Prev_Close']*tmp_df['Shares']),0)

today_pnl_amount = current-previous
today_pnl_percentage = round((current-previous)*100/previous,2)

curr_pnl_amount = current-investment
curr_pnl_percentage = round((current-investment)*100/investment,2)

estimate = round(sum(tmp_df['Max']*tmp_df['Shares']),0)
est_pnl_amount = estimate-investment
est_pnl_percentage = round((estimate-investment)*100/investment,2)

tmp_df['Weightage%'] = round(tmp_df['Investment']*100/investment,2)
tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['Weightage%'],0)

print('Investment: ', investment)
print('Current: ', current)
print('Today PnL: ', today_pnl_amount)
print('Today PnL %: ', today_pnl_percentage)
print('Current PnL: ', curr_pnl_amount)
print('Current PnL %: ', curr_pnl_percentage)
print('Estimate: ', estimate)
print('Estimate PnL: ', est_pnl_amount)
print('Estimate PnL %: ', est_pnl_percentage)

qualified stocks: 35
Investment:  1727255.0
Current:  1778783.0
Today PnL:  -964.0
Today PnL %:  -0.05
Current PnL:  51528.0
Current PnL %:  2.98
Estimate:  3177475.0
Estimate PnL:  1450220.0
Estimate PnL %:  83.96


In [8]:
tmp_df.sort_values(by = 'Current P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
33,612.0,VALIANTORG.NS,1851.0,621.0,530.15,96.0,2T3Y,2T3Y,50894.40,58752.0,59616.0,177696.0,-1.45,15.44,249.15,233.71,2.95,46.0
16,1228.0,ICICIGI.NS,1620.0,1225.0,1106.17,46.0,V40,ATH,50883.82,56488.0,56350.0,74520.0,0.24,11.01,46.45,35.44,2.95,32.0
6,7034.0,BAJFINANCE.NS,7901.0,7040.0,6365.23,16.0,V40,ATH,101843.68,112544.0,112640.0,126416.0,-0.09,10.51,24.13,13.62,5.90,62.0
0,366.0,5PAISA.NS,553.0,368.0,333.09,152.0,V40N,ATH,50629.68,55632.0,55936.0,84056.0,-0.54,9.88,66.02,56.14,2.93,29.0
12,187.0,GICRE.NS,358.0,186.0,171.85,175.0,2T3Y,2T3Y,30073.75,32725.0,32550.0,62650.0,0.54,8.82,108.32,99.50,1.74,15.0
26,2637.0,PIDILITIND.NS,2902.0,2652.0,2442.09,20.0,V40,ATH,48841.80,52740.0,53040.0,58040.0,-0.57,7.98,18.83,10.85,2.83,23.0
31,36.0,UJJIVANSFB.NS,57.0,36.0,33.60,900.0,V40N,ATH,30240.00,32400.0,32400.0,51300.0,0.00,7.14,69.64,62.50,1.75,12.0
21,72.0,JMFINANCIL.NS,121.0,70.0,67.30,450.0,2T3Y,2T3Y,30285.00,32400.0,31500.0,54450.0,2.86,6.98,79.79,72.81,1.75,12.0
3,437.0,AMBUJACEM.NS,588.0,429.0,408.85,120.0,TBD,ATH,49062.00,52440.0,51480.0,70560.0,1.86,6.89,43.82,36.93,2.84,20.0
4,3245.0,ASIANPAINT.NS,3566.0,3241.0,3048.30,17.0,V40,ATH,51821.10,55165.0,55097.0,60622.0,0.12,6.45,16.98,10.53,3.00,19.0


In [9]:
tmp_df.sort_values(by = 'Estimated P/L%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
33,612.0,VALIANTORG.NS,1851.0,621.0,530.15,96.0,2T3Y,2T3Y,50894.40,58752.0,59616.0,177696.0,-1.45,15.44,249.15,233.71,2.95,46.0
14,931.0,GLAND.NS,4316.0,932.0,1354.78,37.0,2T3Y,2T3Y,50126.86,34447.0,34484.0,159692.0,-0.11,-31.28,218.58,249.86,2.90,-91.0
23,1508.0,LUXIND.NS,4546.0,1506.0,1499.87,34.0,V40N,2T3Y,50995.58,51272.0,51204.0,154564.0,0.13,0.54,203.09,202.55,2.95,2.0
32,300.0,VAIBHAVGBL.NS,966.0,301.0,320.64,96.0,2T3Y,2T3Y,30781.44,28800.0,28896.0,92736.0,-0.33,-6.44,201.27,207.71,1.78,-11.0
7,265.0,BANDHANBNK.NS,723.0,268.0,252.96,200.0,2T3Y,2T3Y,50592.00,53000.0,53600.0,144600.0,-1.12,4.76,185.82,181.06,2.93,14.0
24,120.0,NIACL.NS,328.0,120.0,116.60,260.0,TBD,ATH,30316.00,31200.0,31200.0,85280.0,0.00,2.92,181.30,178.38,1.76,5.0
30,2238.0,TEAMLEASE.NS,5281.0,2249.0,2239.86,14.0,V40N,2T3Y,31358.04,31332.0,31486.0,73934.0,-0.49,-0.08,135.77,135.85,1.82,-0.0
1,1393.0,AAVAS.NS,3196.0,1391.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20895.0,20865.0,47940.0,0.14,-0.85,127.47,128.32,1.22,-1.0
17,1468.0,INDIGOPNTS.NS,3112.0,1473.0,1408.25,83.0,V40N,2T3Y,116884.75,121844.0,122259.0,258296.0,-0.34,4.24,120.98,116.74,6.77,29.0
12,187.0,GICRE.NS,358.0,186.0,171.85,175.0,2T3Y,2T3Y,30073.75,32725.0,32550.0,62650.0,0.54,8.82,108.32,99.50,1.74,15.0


In [10]:
tmp_df.sort_values(by = 'Juice Left%', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
4,3245.0,ASIANPAINT.NS,3566.0,3241.0,3048.30,17.0,V40,ATH,51821.10,55165.0,55097.0,60622.0,0.12,6.45,16.98,10.53,3.00,19.0
26,2637.0,PIDILITIND.NS,2902.0,2652.0,2442.09,20.0,V40,ATH,48841.80,52740.0,53040.0,58040.0,-0.57,7.98,18.83,10.85,2.83,23.0
6,7034.0,BAJFINANCE.NS,7901.0,7040.0,6365.23,16.0,V40,ATH,101843.68,112544.0,112640.0,126416.0,-0.09,10.51,24.13,13.62,5.90,62.0
29,2454.0,RELIANCE.NS,2811.0,2463.0,2464.49,21.0,V40,ATH,51754.29,51534.0,51723.0,59031.0,-0.37,-0.43,14.06,14.49,3.00,-1.0
5,1457.0,BAJAJFINSV.NS,1907.0,1450.0,1449.21,35.0,V40,NaN,50722.35,50995.0,50750.0,66745.0,0.48,0.54,31.59,31.05,2.94,2.0
16,1228.0,ICICIGI.NS,1620.0,1225.0,1106.17,46.0,V40,ATH,50883.82,56488.0,56350.0,74520.0,0.24,11.01,46.45,35.44,2.95,32.0
3,437.0,AMBUJACEM.NS,588.0,429.0,408.85,120.0,TBD,ATH,49062.00,52440.0,51480.0,70560.0,1.86,6.89,43.82,36.93,2.84,20.0
25,38700.0,PAGEIND.NS,53549.0,39073.0,39840.50,2.0,V40,ATH,79681.00,77400.0,78146.0,107098.0,-0.95,-2.86,34.41,37.27,4.61,-13.0
8,1570.0,BATAINDIA.NS,2173.0,1578.0,1501.51,67.0,V40,ATH,100601.17,105190.0,105726.0,145591.0,-0.51,4.56,44.72,40.16,5.82,27.0
9,4330.0,BAYERCROP.NS,6002.0,4294.0,4125.00,7.0,V40N,NaN,28875.00,30310.0,30058.0,42014.0,0.84,4.97,45.50,40.53,1.67,8.0


In [11]:
tmp_df.sort_values(by = 'Weightage%', ascending=False)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
17,1468.0,INDIGOPNTS.NS,3112.0,1473.0,1408.25,83.0,V40N,2T3Y,116884.75,121844.0,122259.0,258296.0,-0.34,4.24,120.98,116.74,6.77,29.0
6,7034.0,BAJFINANCE.NS,7901.0,7040.0,6365.23,16.0,V40,ATH,101843.68,112544.0,112640.0,126416.0,-0.09,10.51,24.13,13.62,5.90,62.0
8,1570.0,BATAINDIA.NS,2173.0,1578.0,1501.51,67.0,V40,ATH,100601.17,105190.0,105726.0,145591.0,-0.51,4.56,44.72,40.16,5.82,27.0
25,38700.0,PAGEIND.NS,53549.0,39073.0,39840.50,2.0,V40,ATH,79681.00,77400.0,78146.0,107098.0,-0.95,-2.86,34.41,37.27,4.61,-13.0
13,4450.0,GILLETTE.NS,7529.0,4428.0,4479.47,14.0,V40,ATH,62712.58,62300.0,61992.0,105406.0,0.50,-0.66,68.08,68.74,3.63,-2.0
29,2454.0,RELIANCE.NS,2811.0,2463.0,2464.49,21.0,V40,ATH,51754.29,51534.0,51723.0,59031.0,-0.37,-0.43,14.06,14.49,3.00,-1.0
4,3245.0,ASIANPAINT.NS,3566.0,3241.0,3048.30,17.0,V40,ATH,51821.10,55165.0,55097.0,60622.0,0.12,6.45,16.98,10.53,3.00,19.0
10,3532.0,DMART.NS,5324.0,3501.0,3702.36,14.0,SS,ATH,51833.04,49448.0,49014.0,74536.0,0.89,-4.60,43.80,48.40,3.00,-14.0
33,612.0,VALIANTORG.NS,1851.0,621.0,530.15,96.0,2T3Y,2T3Y,50894.40,58752.0,59616.0,177696.0,-1.45,15.44,249.15,233.71,2.95,46.0
23,1508.0,LUXIND.NS,4546.0,1506.0,1499.87,34.0,V40N,2T3Y,50995.58,51272.0,51204.0,154564.0,0.13,0.54,203.09,202.55,2.95,2.0


In [12]:
tmp_df.to_csv('/content/drive/My Drive/data/stocks/myStocks-Portfolio-Analysis.csv', index=False)

In [14]:
tmp_df.sort_values(by = 'Risk Ind', ascending=True)

,Close,Stock,Max,Prev_Close,Avg_Cost,Shares,Criteria,Strategy,Investment,Current,Previous,Estimated,Today P/L%,Current P/L%,Estimated P/L%,Juice Left%,Weightage%,Risk Ind
14,931.0,GLAND.NS,4316.0,932.0,1354.78,37.0,2T3Y,2T3Y,50126.86,34447.0,34484.0,159692.0,-0.11,-31.28,218.58,249.86,2.90,-91.0
10,3532.0,DMART.NS,5324.0,3501.0,3702.36,14.0,SS,ATH,51833.04,49448.0,49014.0,74536.0,0.89,-4.60,43.80,48.40,3.00,-14.0
25,38700.0,PAGEIND.NS,53549.0,39073.0,39840.50,2.0,V40,ATH,79681.00,77400.0,78146.0,107098.0,-0.95,-2.86,34.41,37.27,4.61,-13.0
32,300.0,VAIBHAVGBL.NS,966.0,301.0,320.64,96.0,2T3Y,2T3Y,30781.44,28800.0,28896.0,92736.0,-0.33,-6.44,201.27,207.71,1.78,-11.0
2,610.0,AMARAJABAT.NS,988.0,621.0,626.83,48.0,V200,ATH,30087.84,29280.0,29808.0,47424.0,-1.77,-2.68,57.62,60.30,1.74,-5.0
13,4450.0,GILLETTE.NS,7529.0,4428.0,4479.47,14.0,V40,ATH,62712.58,62300.0,61992.0,105406.0,0.50,-0.66,68.08,68.74,3.63,-2.0
29,2454.0,RELIANCE.NS,2811.0,2463.0,2464.49,21.0,V40,ATH,51754.29,51534.0,51723.0,59031.0,-0.37,-0.43,14.06,14.49,3.00,-1.0
1,1393.0,AAVAS.NS,3196.0,1391.0,1405.00,15.0,2T3Y,2T3Y,21075.00,20895.0,20865.0,47940.0,0.14,-0.85,127.47,128.32,1.22,-1.0
18,1288.0,INDUSINDBK.NS,1965.0,1276.0,1288.70,24.0,SS,ATH,30928.80,30912.0,30624.0,47160.0,0.94,-0.05,52.48,52.53,1.79,-0.0
30,2238.0,TEAMLEASE.NS,5281.0,2249.0,2239.86,14.0,V40N,2T3Y,31358.04,31332.0,31486.0,73934.0,-0.49,-0.08,135.77,135.85,1.82,-0.0
